In [ ]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

In [13]:
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [14]:
locations = pd.read_csv("locations.csv").to_dict(orient="records")

variables = [
    "temperature_2m", "relative_humidity_2m", "wind_speed_10m", "wind_direction_10m",
    "wind_gusts_10m", "apparent_temperature", "precipitation_probability",
    "precipitation", "rain", "snowfall"
]

all_data = []

In [ ]:
for loc in locations:
    params = {
        "latitude": loc["lat"],
        "longitude": loc["lon"],
        "hourly": variables,
        "hourly_interval": 1,
        "timezone": "Europe/London"
    }

    try:
        responses = openmeteo.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
        response = responses[0]

        hourly = response.Hourly()
        time_index = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        hourly_data = {"time": time_index}
        for i, var in enumerate(variables):
            hourly_data[var] = hourly.Variables(i).ValuesAsNumpy()

        df = pd.DataFrame(hourly_data)
        df["location"] = loc["name"]
        all_data.append(df)

    except Exception as e:
        print(f"Failed to fetch data for {loc['name']}: {e}")

weather_df = pd.concat(all_data, ignore_index=True)

weather_df.to_csv("england_hourly_weather.csv", index=False)
print(weather_df.head())